In [2]:
import functools, os
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
spark = SparkSession.builder.getOrCreate()
sc = SQLContext(spark)

### Data

In [4]:
os.getcwd()

'/sfs/qumulo/qhome/dbn5eu/ds5110/DS5110_MAP21'

#### Input Files

In [5]:
# list inputs
crash_file = "inputs/EPDO Rate.csv"
incident_file = "inputs/LIIR.csv"
ssp_file = "inputs/SSP schedules.csv"
volume_file = "inputs/Vol-VoverC all.csv"
lane_file = "inputs/num lanes.csv"
terrain_file = "inputs/terrain.csv"
truckpct_file = "inputs/Truck Pct.csv"
areatype_file = "inputs/urban_rural.csv"
LOTTR_file = "inputs/rel_unrel.csv"
TMCattribute_file = "inputs/miles urbanCode.csv"
county_file = "inputs/county_district.csv"
TMC_file = "inputs/TMC metadata.csv"
dir_AADT_file = "inputs/Dir AADT.csv"
num_days_file = "inputs/num days in data year.csv"

In [7]:
%%time
# read inputs
crash_data = spark.read.csv(crash_file, header=True)
incident_data = spark.read.csv(incident_file, header=True)
ssp_data = spark.read.csv(ssp_file, header=True)
volume_data = spark.read.csv(volume_file, header=True)
lane_data = spark.read.csv(lane_file, header=True)
terrain_data = spark.read.csv(terrain_file, header=True)
truckpct_data = spark.read.csv(truckpct_file, header=True)
areatype_data = spark.read.csv(areatype_file, header=True)
LOTTR_data = spark.read.csv(LOTTR_file, header=True)
TMCattribute_data = spark.read.csv(TMCattribute_file, header=True)
county_data = spark.read.csv(county_file, header=True)
TMC_data = spark.read.csv(TMC_file, header=True)
dir_AADT_data = spark.read.csv(dir_AADT_file, header=True)
num_days_data = spark.read.csv(num_days_file, header=True)

CPU times: user 14.6 ms, sys: 3.7 ms, total: 18.3 ms
Wall time: 3.85 s


In [8]:
%%time
# merge inputs
sc.sql("set spark.sql.caseSensitive=true")

all_data = TMC_data.join(LOTTR_data, TMC_data.tmc == LOTTR_data.tmc_code, 'left_outer').drop('tmc_code')
all_data = all_data.join(crash_data.withColumnRenamed("Tmc","tmc"), on=['tmc','year'], how='left_outer')
all_data = all_data.join(incident_data.withColumnRenamed("Tmc","tmc"), on=['tmc','year'], how='left_outer')
all_data = all_data.join(ssp_data.withColumnRenamed("Tmc","tmc").withColumnRenamed("Year","year"), on=['tmc','year'], how='left_outer')
all_data = all_data.join(volume_data, on=['tmc','year'], how='left_outer')
all_data = all_data.join(lane_data, on=['tmc','year'], how='left_outer')
all_data = all_data.join(terrain_data, on=['tmc'], how='left_outer')
all_data = all_data.join(truckpct_data, on=['tmc'], how='left_outer')
all_data = all_data.join(areatype_data, on=['tmc'], how='left_outer')
all_data = all_data.join(TMCattribute_data.withColumnRenamed("tmc_code","tmc"), on=['tmc','year'], how='left_outer')
all_data = all_data.join(county_data.withColumnRenamed("tmc_code","tmc"), on=['tmc'], how='left_outer')
all_data = all_data.join(dir_AADT_data.withColumnRenamed("Travel_Time_Code","tmc").withColumnRenamed("Year_Record","year"), on=['tmc','year'], how='left_outer')
all_data = all_data.join(num_days_data.withColumnRenamed("Year","year"), on=['year'], how='left_outer')

CPU times: user 14.7 ms, sys: 578 µs, total: 15.3 ms
Wall time: 1.09 s


#### Fixes

In [9]:
# replace whitespace in column names
all_data = all_data.select([F.col(col).alias(col.replace(' ', '_')) for col in all_data.columns])
print(list(all_data.columns))

['year', 'tmc', 'road', 'dir', 'road_dir', 'Intersection', 'road_order', 'global_road_order', 'obs_AMP_rel_unrel', 'obs_MIDD_rel_unrel', 'obs_PMP_rel_unrel', 'obs_WE_rel_unrel', 'obs_rel_unrel', 'obs_rel_unrel_WD', 'AMP_EPDOR', 'MIDD_EPDOR', 'PMP_EPDOR', 'WE_EPDOR', 'AMP_LIIR', 'MIDD_LIIR', 'PMP_LIIR', 'WE_LIIR', 'AMP_SSP', 'MIDD_SSP', 'PMP_SSP', 'WE_SSP', 'SSP_Periods', 'AMP_Hour_TMS_Vol', 'AMP_TMS_Vol', 'AMP_Lane_TMS_Vol', 'AMP_Hour_Lane_TMS_Vol', 'AMP_Hour_NPMRDS_Vol', 'AMP_NPMRDS_Vol', 'AMP_Lane_NPMRDS_Vol', 'AMP_Hour_Lane_NPMRDS_Vol', 'MIDD_Hour_TMS_Vol', 'MIDD_TMS_Vol', 'MIDD_Lane_TMS_Vol', 'MIDD_Hour_Lane_TMS_Vol', 'MIDD_Hour_NPMRDS_Vol', 'MIDD_NPMRDS_Vol', 'MIDD_Lane_NPMRDS_Vol', 'MIDD_Hour_Lane_NPMRDS_Vol', 'PMP_Hour_TMS_Vol', 'PMP_TMS_Vol', 'PMP_Lane_TMS_Vol', 'PMP_Hour_Lane_TMS_Vol', 'PMP_Hour_NPMRDS_Vol', 'PMP_NPMRDS_Vol', 'PMP_Lane_NPMRDS_Vol', 'PMP_Hour_Lane_NPMRDS_Vol', 'WE_Hour_TMS_Vol', 'WE_TMS_Vol', 'WE_Lane_TMS_Vol', 'WE_Hour_Lane_TMS_Vol', 'WE_Hour_NPMRDS_Vol', 'WE_

In [10]:
# Add occupancy factor and observed LOTTR helper numerator and denominator terms

# default occupancy factor
occ_fac = 1.7

all_data = all_data.withColumn("occ_fac", F.lit(occ_fac))
all_data = all_data.withColumn("obs_isReliable", F.when(all_data['obs_rel_unrel'] == "Rel", 1).otherwise(0))
all_data = all_data.withColumn("obs_LOTTR_helper_denominator", all_data['miles_on_NHS'] * all_data['DIR_AADT'] * all_data['Number_Days'])
all_data = all_data.withColumn("obs_LOTTR_helper_numerator", all_data['obs_LOTTR_helper_denominator'] * all_data['obs_isReliable'])

In [11]:
# Add categorical fields for Terrain, and area type
all_data = all_data.withColumn("Rolling", F.when(all_data['Terrain'] == "Rolling", 1).otherwise(0))
all_data = all_data.withColumn("Urbanized", F.when(all_data['Area_Type'] == "Urbanized", 1).otherwise(0))
all_data = all_data.withColumn("UrbanZCluster", F.when(all_data['Area_Type'] == "Urban Cluster", 1).otherwise(0))

In [12]:
# Drop some columns that are not going to be used
all_data = all_data.drop('road','dir','road_dir','Intersection','county','road order', 'global_road_order', 'Urban_Code','DIR_ADDT','District')

In [13]:
all_data.toPandas()

,year,tmc,road_order,obs_AMP_rel_unrel,obs_MIDD_rel_unrel,obs_PMP_rel_unrel,obs_WE_rel_unrel,obs_rel_unrel,obs_rel_unrel_WD,AMP_EPDOR,...,County,DIR_AADT,Number_Days,occ_fac,obs_isReliable,obs_LOTTR_helper_denominator,obs_LOTTR_helper_numerator,Rolling,Urbanized,UrbanZCluster
0,2020,110+04847,20,Rel,Rel,Rel,Rel,Rel,Rel,11.86970714,...,RICHMOND (CITY),13634,366,1.7,1,8.682677e+05,8.682677e+05,0,1,0
1,2019,110+04847,20,Rel,Rel,Rel,Rel,Rel,Rel,18.7176151,...,RICHMOND (CITY),13144,365,1.7,1,8.443706e+05,8.443706e+05,0,1,0
2,2018,110+04847,20,Rel,Rel,Rel,Rel,Rel,Rel,27.84815905,...,RICHMOND (CITY),12464,365,1.7,1,6.596572e+05,6.596572e+05,0,1,0
3,2017,110+04847,20,Rel,Rel,Rel,Rel,Rel,Rel,16.89150631,...,RICHMOND (CITY),12092,365,1.7,1,6.399691e+05,6.399691e+05,0,1,0
4,2020,110P04847,20.001,Rel,Rel,Rel,Rel,Rel,Rel,11.86970714,...,RICHMOND (CITY),13760,366,1.7,1,1.596463e+06,1.596463e+06,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6823,2017,110N05415,988,Rel,Rel,Rel,Rel,Rel,Rel,1.550820985,...,GREENSVILLE,20992,365,1.7,1,2.972887e+06,2.972887e+06,0,0,0
6824,2020,110-05414,989,Rel,Rel,Rel,Rel,Rel,Rel,1.395738887,...,GREENSVILLE,22170,366,1.7,1,3.272465e+07,3.272465e+07,0,0,0
6825,2019,110-05414,989,Rel,Rel,Rel,Rel,Rel,Rel,0.077541049,...,GREENSVILLE,22509,365,1.7,1,3.313426e+07,3.313426e+07,0,0,0
6826,2018,110-05414,989,Rel,Rel,Rel,Rel,Rel,Rel,13.33706047,...,GREENSVILLE,21552,365,1.7,1,3.189858e+07,3.189858e+07,0,0,0


In [14]:
all_data.toPandas().to_csv("all_data.csv")

#### Reformat Data

The relatively small amount of rows could be an issue with training a classification algorithm. We are going to transform the data from wide to long format by separating all of the period columns into their own distinct dataframes, changing the column names to a universal name, and concatenating the dataframes. We'll then merge on all non-period associated columns. We'll save a copy of the response variable, OBS.rel_unrel, so that after running models we can do a grouby on year, tmc, and period, and compare to the original response format.

In [15]:
# separate data
non_period_cols = ['year', 'tmc', 'obs_rel_unrel', 'lanes', 'miles', 'Rolling',  'Urbanized', 'UrbanZCluster']
AMP_cols = ['year','tmc', 'AMP_EPDOR', 'AMP_LIIR', 'AMP_SSP', 'AMP_Hour_TMS_Vol', 'AMP_TMS_Vol', 'AMP_Lane_TMS_Vol', 'AMP_Hour_Lane_TMS_Vol', 'AMP_Hour_NPMRDS_Vol', 'AMP_NPMRDS_Vol', 'AMP_Hour_Lane_NPMRDS_Vol', 'AMP_NPMRDS_straight_VoverC', 'AMP_TMS_straight_VoverC', 'AMP_NPMRDS_weighted_VoverC', 'AMP_TMS_weighted_VoverC', 'AMP_Truck']
MIDD_cols = ['year','tmc', 'MIDD_EPDOR', 'MIDD_LIIR', 'MIDD_SSP', 'MIDD_Hour_TMS_Vol', 'MIDD_TMS_Vol', 'MIDD_Lane_TMS_Vol', 'MIDD_Hour_Lane_TMS_Vol', 'MIDD_Hour_NPMRDS_Vol', 'MIDD_NPMRDS_Vol', 'MIDD_Hour_Lane_NPMRDS_Vol', 'MIDD_NPMRDS_straight_VoverC', 'MIDD_TMS_straight_VoverC', 'MIDD_NPMRDS_weighted_VoverC', 'MIDD_TMS_weighted_VoverC', 'MIDD_Truck']
PMP_cols = ['year','tmc', 'PMP_EPDOR', 'PMP_LIIR', 'PMP_SSP', 'PMP_Hour_TMS_Vol', 'PMP_TMS_Vol', 'PMP_Lane_TMS_Vol', 'PMP_Hour_Lane_TMS_Vol', 'PMP_Hour_NPMRDS_Vol', 'PMP_NPMRDS_Vol', 'PMP_Hour_Lane_NPMRDS_Vol', 'PMP_NPMRDS_straight_VoverC', 'PMP_TMS_straight_VoverC', 'PMP_NPMRDS_weighted_VoverC', 'PMP_TMS_weighted_VoverC', 'PMP_Truck']
WE_cols = ['year','tmc', 'WE_EPDOR', 'WE_LIIR', 'WE_SSP', 'WE_Hour_TMS_Vol', 'WE_TMS_Vol', 'WE_Lane_TMS_Vol', 'WE_Hour_Lane_TMS_Vol', 'WE_Hour_NPMRDS_Vol', 'WE_NPMRDS_Vol', 'WE_Hour_Lane_NPMRDS_Vol', 'WE_NPMRDS_straight_VoverC', 'WE_TMS_straight_VoverC', 'WE_NPMRDS_weighted_VoverC', 'WE_TMS_weighted_VoverC', 'WE_Truck']
FINAL_TEST_COLS = ['year', 'tmc', 'obs_rel_unrel']

non_period_data = all_data[non_period_cols]
AMP_data = all_data[AMP_cols]
MIDD_data = all_data[MIDD_cols]
PMP_data = all_data[PMP_cols]
WE_data = all_data[WE_cols]
FINAL_TEST_DATA = all_data[FINAL_TEST_COLS]

In [16]:
# rename columns
def renameCols(data, name):
    for col in data.columns:
        data = data.withColumnRenamed(col,col.replace(name,""))
    return data

AMP_data = renameCols(AMP_data,"AMP_")
MIDD_data = renameCols(MIDD_data,"MIDD_")
PMP_data = renameCols(PMP_data,"PMP_")
WE_data = renameCols(WE_data,"WE_")

In [17]:
# add period column
AMP_data = AMP_data.withColumn("Period", F.lit("AMP"))
MIDD_data = MIDD_data.withColumn("Period", F.lit("MIDD"))
PMP_data = PMP_data.withColumn("Period", F.lit("PMP"))
WE_data = WE_data.withColumn("Period", F.lit("WE"))

In [18]:
# concatenate period data
period_data = functools.reduce(lambda df1, df2: df1.union(df2), [AMP_data, MIDD_data, PMP_data, WE_data])

In [19]:
# merge wit the non-period columns
data_final = period_data.join(non_period_data, on=['year','tmc'])

In [20]:
%%time
tempdf = data_final.toPandas()

CPU times: user 298 ms, sys: 28.4 ms, total: 326 ms
Wall time: 33 s


In [21]:
tempdf.isna().sum()

year                         0
tmc                          0
EPDOR                        0
LIIR                         0
SSP                          0
Hour_TMS_Vol              5683
TMS_Vol                   5683
Lane_TMS_Vol              5683
Hour_Lane_TMS_Vol         5683
Hour_NPMRDS_Vol           5683
NPMRDS_Vol                5683
Hour_Lane_NPMRDS_Vol      5683
NPMRDS_straight_VoverC    5684
TMS_straight_VoverC       5684
NPMRDS_weighted_VoverC    5684
TMS_weighted_VoverC       5684
Truck                        0
Period                       0
obs_rel_unrel                0
lanes                        0
miles                        0
Rolling                      0
Urbanized                    0
UrbanZCluster                0
dtype: int64

In [22]:
tempdf

,year,tmc,EPDOR,LIIR,SSP,Hour_TMS_Vol,TMS_Vol,Lane_TMS_Vol,Hour_Lane_TMS_Vol,Hour_NPMRDS_Vol,...,NPMRDS_weighted_VoverC,TMS_weighted_VoverC,Truck,Period,obs_rel_unrel,lanes,miles,Rolling,Urbanized,UrbanZCluster
0,2017,110+04872,19.63565634,3.813536562,Y,29.2991477,117.1965908,58.5982954,14.64957385,29.36019555,...,0.686666554,0.685238788,4.2172235,AMP,Unrel,2,0.625677,0,1,0
1,2017,110+04872,21.58299416,9.980106321,Y,25.78331757,154.6999054,77.3499527,12.89165878,25.8370398,...,0.605835062,0.604575366,4.4685001,MIDD,Unrel,2,0.625677,0,1,0
2,2017,110+04872,21.17729878,7.545934048,Y,25.22079243,100.8831697,50.44158485,12.61039621,25.27334258,...,0.582412502,0.581201508,1.3178263,PMP,Unrel,2,0.625677,0,1,0
3,2017,110+04872,8.84415926,7.870490351,Y,23.29872912,372.7796659,186.389833,11.64936456,23.34727446,...,0.624908616,0.623609261,3.536854,WE,Unrel,2,0.625677,0,1,0
4,2017,110-05363,0.293321904,0.146660952,Y,11.80428475,47.21713899,23.6085695,5.902142374,11.19204972,...,0.370558037,0.390828552,22.4799893,AMP,Rel,2,6.559198,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27307,2020,110P05355,1.787366149,0.487463495,Y,None,None,None,None,None,...,None,None,24.2774236,WE,Rel,2,0.580561,1,0,0
27308,2020,110P05518,0.664139542,0.110689924,Y,None,None,None,None,None,...,None,None,35.1987437,AMP,Rel,2,0.512301,1,0,0
27309,2020,110P05518,0.332069771,1.771038778,Y,None,None,None,None,None,...,None,None,32.0038392,MIDD,Rel,2,0.512301,1,0,0
27310,2020,110P05518,0.332069771,1.328279084,Y,None,None,None,None,None,...,None,None,35.7868416,PMP,Rel,2,0.512301,1,0,0


In [23]:
tempdf.to_csv("data_long.csv")